# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

import matplotlib.pyplot as plt

from torch import cuda

from absa_models.ABSA_BERT_Dropout_Linear import ABSA_BERT_Dropout_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ABSA_SemEval16_Restaurants_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,absa_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8
SEQ_LEN = 512

NO_RUNS = 10

In [10]:
BERT_FINE_TUNED_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_512.pth'

In [11]:
MODEL_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/models/bert_pre_trained_dropout_linear_512.pth'
STATS_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/stats/bert_pre_trained_dropout_linear_512.csv'

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
best_accuracy = 0.0

In [14]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(ids_tensors[0])), 0)(ids_tensors[0])
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(tags_tensors[0])), 0)(tags_tensors[0])
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [15]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    losses = []

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 4), tags_tensors.view(-1))

        losses.append(loss.item())
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()
    
    return losses

In [16]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [17]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [18]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ABSA_BERT_Dropout_Linear(BertModel.from_pretrained('bert-base-uncased'), dropout=0.3, no_out_labels=4, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    train_losses = []

    for epoch in range(EPOCHS):
        losses = train(epoch, model, loss_fn, optimizer, train_dataloader)

        train_losses += losses
    
    plt.title(f'Train Loss for run {i + 1}/{NO_RUNS}')
    plt.plot(train_losses)
    plt.savefig(f'../../../results/ABSA/SemEval16 - Task 5 - Restaurants/plots/bert_pt_do_lin/train_loss_run_{i + 1}.png')

    plt.clf()

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.bert, BERT_FINE_TUNED_OUTPUT)
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.6813029050827026
Epoch: 0/2, Batch: 50/501, Loss: 0.032318178564310074
Epoch: 0/2, Batch: 100/501, Loss: 0.023857953026890755
Epoch: 0/2, Batch: 150/501, Loss: 0.026729945093393326
Epoch: 0/2, Batch: 200/501, Loss: 0.011068263091146946
Epoch: 0/2, Batch: 250/501, Loss: 0.022189825773239136
Epoch: 0/2, Batch: 300/501, Loss: 0.019006170332431793
Epoch: 0/2, Batch: 350/501, Loss: 0.03243911266326904
Epoch: 0/2, Batch: 400/501, Loss: 0.019130604341626167
Epoch: 0/2, Batch: 450/501, Loss: 0.01511656865477562
Epoch: 0/2, Batch: 500/501, Loss: 0.009167409501969814
Epoch: 1/2, Batch: 0/501, Loss: 0.01789938658475876
Epoch: 1/2, Batch: 50/501, Loss: 0.02422196976840496
Epoch: 1/2, Batch: 100/501, Loss: 0.010774345137178898
Epoch: 1/2, Batch: 150/501, Loss: 0.012576853856444359
Epoch: 1/2, Batch: 200/501, Loss: 0.004896908067166805
Epoch: 1/2, Batch: 250/501, Loss: 0.006164856720715761
Epoch: 1/2, Batch: 300/501, Loss: 0.01636437140405178
Epoch: 1/2, Ba

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 2/10
Epoch: 0/2, Batch: 0/501, Loss: 1.2682029008865356
Epoch: 0/2, Batch: 50/501, Loss: 0.028405262157320976
Epoch: 0/2, Batch: 100/501, Loss: 0.018584750592708588
Epoch: 0/2, Batch: 150/501, Loss: 0.02536383457481861
Epoch: 0/2, Batch: 200/501, Loss: 0.009203222580254078
Epoch: 0/2, Batch: 250/501, Loss: 0.016501735895872116
Epoch: 0/2, Batch: 300/501, Loss: 0.01723845861852169
Epoch: 0/2, Batch: 350/501, Loss: 0.013724899850785732
Epoch: 0/2, Batch: 400/501, Loss: 0.014708678238093853
Epoch: 0/2, Batch: 450/501, Loss: 0.01828308403491974
Epoch: 0/2, Batch: 500/501, Loss: 0.0071647511795163155
Epoch: 1/2, Batch: 0/501, Loss: 0.01606978289783001
Epoch: 1/2, Batch: 50/501, Loss: 0.011026180349290371
Epoch: 1/2, Batch: 100/501, Loss: 0.013074729591608047
Epoch: 1/2, Batch: 150/501, Loss: 0.009941943921148777
Epoch: 1/2, Batch: 200/501, Loss: 0.016430707648396492
Epoch: 1/2, Batch: 250/501, Loss: 0.006025110371410847
Epoch: 1/2, Batch: 300/501, Loss: 0.004438766278326511
Epoch: 1/2, 

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/501, Loss: 1.3954671621322632
Epoch: 0/2, Batch: 50/501, Loss: 0.09592089056968689
Epoch: 0/2, Batch: 100/501, Loss: 0.028878364711999893
Epoch: 0/2, Batch: 150/501, Loss: 0.04011870175600052
Epoch: 0/2, Batch: 200/501, Loss: 0.03877433389425278
Epoch: 0/2, Batch: 250/501, Loss: 0.03684636950492859
Epoch: 0/2, Batch: 300/501, Loss: 0.03523527458310127
Epoch: 0/2, Batch: 350/501, Loss: 0.02258038893342018
Epoch: 0/2, Batch: 400/501, Loss: 0.007161418441683054
Epoch: 0/2, Batch: 450/501, Loss: 0.019068334251642227
Epoch: 0/2, Batch: 500/501, Loss: 0.012949289754033089
Epoch: 1/2, Batch: 0/501, Loss: 0.011499855667352676
Epoch: 1/2, Batch: 50/501, Loss: 0.021116849035024643
Epoch: 1/2, Batch: 100/501, Loss: 0.007814641110599041
Epoch: 1/2, Batch: 150/501, Loss: 0.026213373988866806
Epoch: 1/2, Batch: 200/501, Loss: 0.01370814349502325
Epoch: 1/2, Batch: 250/501, Loss: 0.022534247487783432
Epoch: 1/2, Batch: 300/501, Loss: 0.009343501180410385
Epoch: 1/2, Batc

...

Run 9/10
Epoch: 0/2, Batch: 0/501, Loss: 1.401513934135437
Epoch: 0/2, Batch: 50/501, Loss: 0.03550565987825394
Epoch: 0/2, Batch: 100/501, Loss: 0.03808692842721939
Epoch: 0/2, Batch: 150/501, Loss: 0.011559308506548405
Epoch: 0/2, Batch: 200/501, Loss: 0.037529561668634415
Epoch: 0/2, Batch: 250/501, Loss: 0.027028538286685944
Epoch: 0/2, Batch: 300/501, Loss: 0.013810166157782078
Epoch: 0/2, Batch: 350/501, Loss: 0.017548993229866028
Epoch: 0/2, Batch: 400/501, Loss: 0.024630781263113022
Epoch: 0/2, Batch: 450/501, Loss: 0.012560867704451084
Epoch: 0/2, Batch: 500/501, Loss: 0.012194148264825344
Epoch: 1/2, Batch: 0/501, Loss: 0.014036959037184715
Epoch: 1/2, Batch: 50/501, Loss: 0.008247597143054008
Epoch: 1/2, Batch: 100/501, Loss: 0.007078836672008038
Epoch: 1/2, Batch: 150/501, Loss: 0.013338840566575527
Epoch: 1/2, Batch: 200/501, Loss: 0.012130413204431534
Epoch: 1/2, Batch: 250/501, Loss: 0.008928400464355946
Epoch: 1/2, Batch: 300/501, Loss: 0.006748026702553034
Epoch: 1/2, 

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 10/10
Epoch: 0/2, Batch: 0/501, Loss: 1.522618055343628
Epoch: 0/2, Batch: 50/501, Loss: 0.0243550855666399
Epoch: 0/2, Batch: 100/501, Loss: 0.04350343719124794
Epoch: 0/2, Batch: 150/501, Loss: 0.02152916043996811
Epoch: 0/2, Batch: 200/501, Loss: 0.023712893947958946
Epoch: 0/2, Batch: 250/501, Loss: 0.029804641380906105
Epoch: 0/2, Batch: 300/501, Loss: 0.034557946026325226
Epoch: 0/2, Batch: 350/501, Loss: 0.035111717879772186
Epoch: 0/2, Batch: 400/501, Loss: 0.02310279756784439
Epoch: 0/2, Batch: 450/501, Loss: 0.010644391179084778
Epoch: 0/2, Batch: 500/501, Loss: 0.013952825218439102
Epoch: 1/2, Batch: 0/501, Loss: 0.013465547002851963
Epoch: 1/2, Batch: 50/501, Loss: 0.017365435138344765
Epoch: 1/2, Batch: 100/501, Loss: 0.009584645740687847
Epoch: 1/2, Batch: 150/501, Loss: 0.012984356842935085
Epoch: 1/2, Batch: 200/501, Loss: 0.013406788930296898
Epoch: 1/2, Batch: 250/501, Loss: 0.021128738299012184
Epoch: 1/2, Batch: 300/501, Loss: 0.0054162899032235146
Epoch: 1/2, B

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<Figure size 432x288 with 0 Axes>

In [19]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.996875,0.996875,0.400640,0.996875,0.450754,0.996875,0.422131,331.585561
1,0.997449,0.997449,0.407401,0.997449,0.466177,0.997449,0.432167,329.424239
2,0.997289,0.997289,0.410107,0.997289,0.463625,0.997289,0.433034,330.981003
3,0.997453,0.997453,0.608813,0.997453,0.479246,0.997453,0.455660,330.131548
4,0.997223,0.997223,0.400973,0.997223,0.467495,0.997223,0.428216,330.963369
5,0.997523,0.997523,0.423359,0.997523,0.448171,0.997523,0.434948,361.233716
6,0.997371,0.997371,0.415364,0.997371,0.472753,0.997371,0.439804,356.150496
7,0.997117,0.997117,0.405068,0.997117,0.444003,0.997117,0.422382,347.421640
8,0.997426,0.997426,0.408809,0.997426,0.463880,0.997426,0.432274,342.439443
9,0.997195,0.997195,0.398292,0.997195,0.474628,0.997195,0.428631,339.185600


In [20]:
results.to_csv(STATS_OUTPUT)